In [2]:
#!pip install torch pandas numpy h5py tqdm scikit-learn tensorboard

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.7 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 2.7 MB/s eta 0:00:00a 0:00:01


In [3]:
%load_ext autoreload
%autoreload 2

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import joblib
from src.dataset import ProteinDataset
from src.utils import train_model, test_model
import torch
from src.model import ChemicalShiftsPredictor, ChemicalShiftsPredictorAttention

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ModuleNotFoundError: No module named 'tensorboard'

In [21]:
# Load and prepare data
csv_file = 'data/strict.csv'
prott5_file = 'data/embeddings/unfiltered_all_prott5.h5'
prott5_res_file = 'data/embeddings/unfiltered_all_prott5_res.h5'
prostt5_file = 'data/embeddings/prostt5.h5'
esm_file = 'data/embeddings/unfiltered_all_esm2_3b.h5'
esm_res_file = 'data/embeddings/unfiltered_all_esm2_3b_res.h5'
chemical_shifts_df = pd.read_csv(csv_file)
#chemical_shifts_df.describe()

In [22]:
test_ids = []
with open("pdb_matched/final_test_ids.txt", "r") as f:
    for line in f:
        test_ids.append(line.strip())

In [23]:
chemical_shifts_df = chemical_shifts_df[chemical_shifts_df['ID'].isin(test_ids)]

In [24]:
scaler_applied = False

In [25]:
chemical_shifts_df

,Unnamed: 0,ID,entryID,stID,entity_assemID,entityID,seq_index,seq,k,zscores,pscores,C,CA,CB,HA,H,N,HB
447,6,30161_1_1_1,30161,1,1,1,1,M,7,NaN,NaN,175.986,55.600,33.138,4.401,NaN,121.816,1.9350
448,6,30161_1_1_1,30161,1,1,1,2,I,14,11.1609,0.0947,174.316,61.588,38.193,3.830,8.107,123.431,1.7160
449,6,30161_1_1_1,30161,1,1,1,3,R,21,14.1291,0.0736,176.156,57.343,32.755,4.716,9.806,127.627,1.8845
450,6,30161_1_1_1,30161,1,1,1,4,T,21,15.1705,0.0332,173.925,58.964,71.096,5.291,8.631,110.835,4.3560
451,6,30161_1_1_1,30161,1,1,1,5,I,21,14.7280,0.0497,173.317,59.542,43.057,4.875,8.368,120.009,1.2950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
215543,1893,36334_1_1_1,36334,1,1,1,106,H,1,3.8761,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215544,1893,36334_1_1_1,36334,1,1,1,107,H,2,2.3427,0.4918,NaN,NaN,NaN,NaN,NaN,NaN,NaN
215545,1893,36334_1_1_1,36334,1,1,1,108,H,2,2.3427,0.4918,NaN,NaN,30.053,NaN,8.212,NaN,NaN
215546,1893,36334_1_1_1,36334,1,1,1,109,H,2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [26]:
#target_columns = ['C', 'CA', 'CB', 'HA', 'H', 'N', 'HB']
target_columns = ['N']
chemical_shifts_df.dropna(inplace=True, subset=target_columns)


scaler = joblib.load('scaler.joblib')
means = scaler.mean_
stds = scaler.scale_

# Apply normalization to the training targets
#train_df[target_columns] = scaler.transform(train_targets)

# Apply the same normalization to validation and test sets
# val_df[target_columns] = scaler.transform(val_df[target_columns])
# test_df[target_columns] = scaler.transform(test_df[target_columns])

# Create datasets
if not scaler_applied:
    chemical_shifts_df[target_columns] = scaler.transform(chemical_shifts_df[target_columns])
    scaler_applied = True
test_dataset = ProteinDataset(target_columns, chemical_shifts_df, prott5_file, prott5_res_file, prostt5_file, esm_res_file, esm_file)

In [27]:
print('Test dataset length:', len(test_dataset))

Test dataset length: 11531


In [28]:
learning_rate = 0.001
weight_decay = 1e-5
patience = 10
batch_size = 128
num_epochs = 5

use_prostt5 = True
use_protein_mean = True
use_attention = True

In [29]:
torch.cuda.is_available() 

False

In [12]:
model = ChemicalShiftsPredictor(use_prostt5=use_prostt5, use_protein_mean=use_protein_mean, use_attention=use_attention).to("cpu")
model.load_state_dict(torch.load('Full_1e-4.pth', map_location=torch.device('cpu')))

<All keys matched successfully>

In [13]:
test_model(model, test_dataset, batch_size=batch_size, use_prostt5=use_prostt5, use_protein_mean=use_protein_mean, use_attention=use_attention, reinit_model=False, scaler=scaler)

  0%|          | 0/91 [00:00<?, ?it/s]

100%|██████████| 91/91 [03:30<00:00,  2.32s/it]

Test Loss: 0.3524
Test RMSE: 2.9973


In [14]:
import os

ucb_predictions = {}

for filename in os.listdir("output_ucb"):
    file_path = os.path.join("output_ucb", filename)
    id = filename.split(';')[0]
    ucb_predictions[id] = pd.read_csv(file_path)
    break

In [18]:
three_to_one = {
    'ALA': 'A', 'ARG': 'R', 'ASN': 'N', 'ASP': 'D', 'CYS': 'C', 
    'GLU': 'E', 'GLN': 'Q', 'GLY': 'G', 'HIS': 'H', 'ILE': 'I', 
    'LEU': 'L', 'LYS': 'K', 'MET': 'M', 'PHE': 'F', 'PRO': 'P', 
    'SER': 'S', 'THR': 'T', 'TRP': 'W', 'TYR': 'Y', 'VAL': 'V',
}

def process_ucb_output(dataframe, id):
    new_columns = {}
    new_columns["ID"] = id
    new_columns['seq_index'] = dataframe["RESNUM"] - max(dataframe["RESNUM"]) + 1
    new_columns['seq'] = [three_to_one[res] for res in dataframe["RESNAME"]]
    new_columns['H'] = dataframe["H_UCBshift"]
    new_columns['N'] = dataframe["N_UCBshift"]

[autoreload of psutil failed: Traceback (most recent call last):
  File "/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/sergeyer/mambaforge/envs/prots/lib/python3.12/importlib/__init__.py", line 131, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 866, in _exec
  File "<frozen importlib._bootstrap_external>", line 994, in exec_module
  File "<frozen importlib._bootstrap>", line 488, in _call_with_frames_removed
  File "/home/sergeyer/mambaforge/envs/prots/lib/python3.12/site-packages/psutil/__init__.py", line 250, in <module>
    raise ImportError(msg)
ImportError: version conflict: '/home/sergeyer/mambaforge/envs/prots/lib/pyth

In [16]:
ucb_predictions['34695_1_1_1']

,RESNUM,RESNAME,H_X,H_Y,H_BEST_REF_SCORE,H_BEST_REF_COV,H_BEST_REF_MATCH,H_UCBShift,HA_X,HA_Y,...,CB_BEST_REF_SCORE,CB_BEST_REF_COV,CB_BEST_REF_MATCH,CB_UCBShift,N_X,N_Y,N_BEST_REF_SCORE,N_BEST_REF_COV,N_BEST_REF_MATCH,N_UCBShift
0,-2,SER,8.444545,NaN,0,0,0,8.444545,4.314632,NaN,...,0,0,0,64.048169,117.078153,NaN,0,0,0,117.078153
1,-1,GLY,8.411867,NaN,0,0,0,8.411867,3.919937,NaN,...,0,0,0,NaN,112.167483,NaN,0,0,0,112.167483
2,0,GLY,8.140765,NaN,0,0,0,8.140765,3.953934,NaN,...,0,0,0,NaN,111.149962,NaN,0,0,0,111.149962
3,1,ALA,6.795665,NaN,0,0,0,6.795665,4.567360,NaN,...,0,0,0,22.227971,122.200070,NaN,0,0,0,122.200070
4,2,LEU,9.064683,NaN,0,0,0,9.064683,5.033738,NaN,...,0,0,0,43.809984,119.591293,NaN,0,0,0,119.591293
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135,133,PHE,8.106282,NaN,0,0,0,8.106282,4.446982,NaN,...,0,0,0,39.943547,117.901220,NaN,0,0,0,117.901220
136,134,ILE,7.611105,NaN,0,0,0,7.611105,4.292674,NaN,...,0,0,0,37.226210,120.370541,NaN,0,0,0,120.370541
137,135,ALA,8.836116,NaN,0,0,0,8.836116,3.600225,NaN,...,0,0,0,20.511458,131.937400,NaN,0,0,0,131.937400
138,136,SER,7.299735,NaN,0,0,0,7.299735,4.791881,NaN,...,0,0,0,64.746134,114.903284,NaN,0,0,0,114.903284


In [ ]:
torch.cuda.empty_cache()
trained_model = train_model(train_dataset, val_dataset, learning_rate=learning_rate, num_epochs=50, weight_decay=weight_decay, patience=patience, batch_size=2048, use_prostt5=True, use_protein_mean=True, use_esm2=False)
test_model(trained_model, test_dataset, batch_size=batch_size, use_prostt5=True, use_protein_mean=True, use_esm2=False)